### Importando bibliotecas

In [1]:
# desabilita os warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

### Importando bases de dados

In [ ]:
df_solar_wind = pd.read_csv(r'..\data\solar_wind.csv', usecols=['period',
                                                                'timedelta',
                                                                'bx_gse',
                                                                'by_gse',
                                                                'bz_gse',
                                                                'bt',
                                                                'density',
                                                                'speed',
                                                                'temperature',
                                                                'source'] )
df_solar_wind.head(3)

In [ ]:
df_satellite_pos = pd.read_csv(r'..\data\satellite_pos.csv')
df_satellite_pos.head(3)

In [ ]:
df_sunspots = pd.read_csv(r'..\data\sunspots.csv')
df_sunspots.head(3)

In [ ]:
df_labels = pd.read_csv(r'..\data\labels.csv')
df_labels.head(3)

### Tratando tipos de dados

##### Tratando tipos de dados do df_solar_wind

In [ ]:
df_solar_wind.info()

In [ ]:
# Otimizando tipos de dados das colunas numéricas
col_num_solar_wind = df_solar_wind.select_dtypes(include='float64').columns
df_solar_wind[col_num_solar_wind] = (df_solar_wind[col_num_solar_wind]
                                     .apply(pd.to_numeric, 
                                            downcast='float'))

# Corrigindo tipo de dado da coluna timedelta
df_solar_wind["timedelta"] = pd.to_timedelta(df_solar_wind["timedelta"])

# Corrigindo tipo de dado das colunas categoricas
col_cat_solar_wind = df_solar_wind.select_dtypes(include='object').columns
df_solar_wind[col_cat_solar_wind] = (df_solar_wind[col_cat_solar_wind]
                                     .astype('category'))
df_solar_wind.info()

##### Tratando tipos de dados do df_satellite_pos

In [ ]:
df_satellite_pos.info()

In [ ]:
# Otimizando tipos de dados das colunas numéricas
col_num_satellite_pos = df_satellite_pos.select_dtypes(include='float64').columns
df_satellite_pos[col_num_satellite_pos] = df_satellite_pos[col_num_satellite_pos].apply(pd.to_numeric, downcast='float')

# Corrigindo tipo de dado da coluna timedelta
df_satellite_pos["timedelta"] = pd.to_timedelta(df_satellite_pos["timedelta"])

# col_cat_satellite_pos = df_satellite_pos.select_dtypes(include='object').columns
# df_satellite_pos[col_cat_satellite_pos] = df_solar_wind[col_cat_satellite_pos].astype('category')
df_satellite_pos["period"] = df_satellite_pos["period"].astype('category')
df_satellite_pos.info()

##### Tratando tipos de dados do df_sunspots

In [ ]:
df_sunspots.info()

In [ ]:
# otimizando tipos de dados das colunas numéricas
col_num_sunspots = df_sunspots.select_dtypes(include='float64').columns
df_sunspots[col_num_sunspots] = df_sunspots[col_num_sunspots].apply(pd.to_numeric, downcast='float')

# Corrigindo tipo de dado da coluna timedelta
df_sunspots["timedelta"] = pd.to_timedelta(df_sunspots["timedelta"])

# Corrigindo tipo de dado da coluna period
df_sunspots["period"] = df_sunspots["period"].astype('category')
df_sunspots.info()

##### Tratando tipos de dados do df_labels

In [ ]:
df_labels.info()

In [ ]:
# otimizando tipos de dados das colunas numéricas
col_num_labels = df_labels.select_dtypes(include='int64').columns
df_labels[col_num_labels] = df_labels[col_num_labels].apply(pd.to_numeric, downcast='integer')

# Corrigindo tipo de dado da coluna timedelta
df_labels["timedelta"] = pd.to_timedelta(df_labels["timedelta"])

# Corrigindo tipo de dado da coluna period
df_labels["period"] = df_labels["period"].astype('category')
df_labels.info()

### Removendo Valores fora de escala do df_solar_wind

In [ ]:
# bx_gse: -200 a +200 nT ok
# by_gse: -200 a +200 nT ok
# bz_gse: -200 a +200 nT ok
# bt: 0 a 200 nT ok
# density: 0 a 200 N/cm³ ok 
# speed: 200 a 2.000 km/s speed=0?
# temperature: 10.000 K a 10.000.000 K temperature=0?
df_solar_wind.describe()

In [46]:
df_solar_wind["speed"].loc[(df_solar_wind["speed"] < 200) | (df_solar_wind["speed"] > 2000)] = np.nan
df_solar_wind["temperature"].loc[(df_solar_wind["temperature"] < 10_000) | (df_solar_wind["temperature"] > 10_000_000)] = np.nan

### Preenchendo fontes ausentes do df_solar_wind com a moda de cada período

In [ ]:
df_solar_wind["source"].isna().sum()

In [ ]:
df_solar_wind_period_a = (
    df_solar_wind.loc[df_solar_wind["period"] == "train_a"])

df_solar_wind_period_b = (
    df_solar_wind.loc[df_solar_wind["period"] == "train_b"])

df_solar_wind_period_c = (
    df_solar_wind.loc[df_solar_wind["period"] == "train_c"])

moda_a = df_solar_wind_period_a["source"].mode()[0]
moda_b = df_solar_wind_period_b["source"].mode()[0]
moda_c = df_solar_wind_period_c["source"].mode()[0]

df_solar_wind_period_a["source"].fillna(moda_a, inplace=True)
df_solar_wind_period_b["source"].fillna(moda_b, inplace=True)
df_solar_wind_period_c["source"].fillna(moda_c, inplace=True)

df_solar_wind = pd.concat(
    [df_solar_wind_period_a, 
     df_solar_wind_period_b, 
     df_solar_wind_period_c], 
     ignore_index=True)

df_solar_wind.head(3)

### Consolidando base de dados

##### Preparando df_solar_wind e df_satellite_pos

In [49]:
df_solar_wind["dia"] = df_solar_wind["timedelta"].dt.days
df_satellite_pos["dia"] = df_satellite_pos["timedelta"].dt.days

In [50]:
df_satellite_pos = pd.merge(df_satellite_pos, 
                            df_solar_wind[["period", "dia", "source"]], 
                            on=["period", "dia"], 
                            how="left")

In [51]:
df_satellite_pos["gse_x_satellite"] = np.nan
df_satellite_pos["gse_y_satellite"] = np.nan
df_satellite_pos["gse_z_satellite"] = np.nan

df_satellite_pos["gse_x_satellite"] = np.where(
    df_satellite_pos["gse_x_satellite"].isnull() 
    & (df_satellite_pos["source"]  == "ac"),
    df_satellite_pos["gse_x_ace"],
    df_satellite_pos["gse_x_dscovr"])

df_satellite_pos["gse_y_satellite"] = np.where(
    df_satellite_pos["gse_y_satellite"].isnull() 
    & (df_satellite_pos["source"]  == "ac"),
    df_satellite_pos["gse_y_ace"],
    df_satellite_pos["gse_y_dscovr"])

df_satellite_pos["gse_z_satellite"] = np.where(
    df_satellite_pos["gse_z_satellite"].isnull() 
    & (df_satellite_pos["source"]  == "ac"),
    df_satellite_pos["gse_z_ace"],
    df_satellite_pos["gse_z_dscovr"])

df_satellite_pos = df_satellite_pos[['period', 
                                     'timedelta', 
                                     'dia', 
                                     'source',
                                     'gse_x_satellite', 
                                     'gse_y_satellite', 
                                     'gse_z_satellite']]

##### Consolidando df_solar_wind e df_satellite_pos

In [ ]:
df_solar_wind["gse_x_satellite"] = np.nan
df_solar_wind["gse_y_satellite"] = np.nan
df_solar_wind["gse_z_satellite"] = np.nan

df_solar_wind["gse_x_satellite"] = np.where(
    df_solar_wind["gse_x_satellite"].isna() 
    & (df_solar_wind["source"]  == df_satellite_pos["source"])
    & (df_solar_wind["period"]  == df_satellite_pos["period"])
    & (df_solar_wind["dia"]  == df_satellite_pos["dia"]),
    df_satellite_pos["gse_x_satellite"],
    np.nan)

df_solar_wind["gse_y_satellite"] = np.where(
    df_solar_wind["gse_y_satellite"].isna() 
    & (df_solar_wind["source"]  == df_satellite_pos["source"])
    & (df_solar_wind["period"]  == df_satellite_pos["period"])
    & (df_solar_wind["dia"]  == df_satellite_pos["dia"]),
    df_satellite_pos["gse_y_satellite"],
    np.nan)

df_solar_wind["gse_z_satellite"] = np.where(
    df_solar_wind["gse_z_satellite"].isna() 
    & (df_solar_wind["source"]  == df_satellite_pos["source"])
    & (df_solar_wind["period"]  == df_satellite_pos["period"])
    & (df_solar_wind["dia"]  == df_satellite_pos["dia"]),
    df_satellite_pos["gse_z_satellite"],
    np.nan)

df_solar_wind.head(3)

##### Consolidando df_solar_wind e df_sunspots

In [ ]:
df_solar_wind = pd.merge(df_solar_wind, 
                         df_sunspots, 
                         on=['period', 'timedelta'], 
                         how='left')
df_solar_wind.head(3)

##### Consolidando df_solar_wind e df_labels

In [ ]:
df_solar_wind = pd.merge(df_solar_wind, 
                         df_labels, 
                         on=['period', 'timedelta'], 
                         how='left')
df_solar_wind.head(3)

##### Removendo coluna dia

In [ ]:
df_solar_wind.drop("dia", axis=1, inplace=True)
df_solar_wind.head(3)

##### Exportando base de dados consolidada

In [61]:
df_solar_wind.to_parquet(r"..\data\df_consolidado.parquet")

### Preenchendo valores ausentes para base de dados de desenvolvimento

##### Segmentando base de dados por período

In [55]:
df_desenvolvimento = df_solar_wind.copy()

df_desenvolvimento_period_a = (
    df_desenvolvimento.loc[df_desenvolvimento["period"] == "train_a"])

df_desenvolvimento_period_b = (
    df_desenvolvimento.loc[df_desenvolvimento["period"] == "train_b"])

df_desenvolvimento_period_c = (
    df_desenvolvimento.loc[df_desenvolvimento["period"] == "train_c"])

##### Preenchendo valores ausentes dos vendos solares com interpolação linear

In [56]:
df_desenvolvimento_period_a["bx_gse"] = (
    df_desenvolvimento_period_a["bx_gse"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_a["by_gse"] = (
    df_desenvolvimento_period_a["by_gse"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_a["bz_gse"] = (
    df_desenvolvimento_period_a["bz_gse"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_a["bt"] = (
    df_desenvolvimento_period_a["bt"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_a["density"] = (
    df_desenvolvimento_period_a["density"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_a["speed"] = (
    df_desenvolvimento_period_a["speed"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_a["temperature"] = (
    df_desenvolvimento_period_a["temperature"]
    .interpolate(method='linear', limit_direction='both'))

df_desenvolvimento_period_b["bx_gse"] = (
    df_desenvolvimento_period_b["bx_gse"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_b["by_gse"] = (
    df_desenvolvimento_period_b["by_gse"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_b["bz_gse"] = (
    df_desenvolvimento_period_b["bz_gse"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_b["bt"] = (
    df_desenvolvimento_period_b["bt"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_b["density"] = (
    df_desenvolvimento_period_b["density"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_b["speed"] = (
    df_desenvolvimento_period_b["speed"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_b["temperature"] = (
    df_desenvolvimento_period_b["temperature"]
    .interpolate(method='linear', limit_direction='both'))

df_desenvolvimento_period_c["bx_gse"] = (
    df_desenvolvimento_period_c["bx_gse"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_c["by_gse"] = (
    df_desenvolvimento_period_c["by_gse"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_c["bz_gse"] = (
    df_desenvolvimento_period_c["bz_gse"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_c["bt"] = (
    df_desenvolvimento_period_c["bt"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_c["density"] = (
    df_desenvolvimento_period_c["density"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_c["speed"] = (
    df_desenvolvimento_period_c["speed"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_c["temperature"] = (
    df_desenvolvimento_period_c["temperature"]
    .interpolate(method='linear', limit_direction='both'))

##### Preenchendo valores ausentes das posições dos satélites com interpolação linear

In [57]:
df_desenvolvimento_period_a["gse_x_satellite"] = (
    df_desenvolvimento_period_a["gse_x_satellite"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_a["gse_x_satellite"] = (
    df_desenvolvimento_period_a["gse_x_satellite"]
    .apply(pd.to_numeric, downcast ="float"))

df_desenvolvimento_period_a["gse_y_satellite"] = (
    df_desenvolvimento_period_a["gse_y_satellite"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_a["gse_y_satellite"] = (
    df_desenvolvimento_period_a["gse_y_satellite"]
    .apply(pd.to_numeric, downcast ="float"))

df_desenvolvimento_period_a["gse_z_satellite"] = (
    df_desenvolvimento_period_a["gse_z_satellite"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_a["gse_z_satellite"] = (
    df_desenvolvimento_period_a["gse_z_satellite"]
    .apply(pd.to_numeric, downcast ="float"))

df_desenvolvimento_period_b["gse_x_satellite"] = (
    df_desenvolvimento_period_b["gse_x_satellite"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_b["gse_x_satellite"] = (
    df_desenvolvimento_period_b["gse_x_satellite"]
    .apply(pd.to_numeric, downcast ="float"))

df_desenvolvimento_period_b["gse_y_satellite"] = (
    df_desenvolvimento_period_b["gse_y_satellite"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_b["gse_y_satellite"] = (
    df_desenvolvimento_period_b["gse_y_satellite"]
    .apply(pd.to_numeric, downcast ="float"))

df_desenvolvimento_period_b["gse_z_satellite"] = (
    df_desenvolvimento_period_b["gse_z_satellite"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_b["gse_z_satellite"] = (
    df_desenvolvimento_period_b["gse_z_satellite"]
    .apply(pd.to_numeric, downcast ="float"))

df_desenvolvimento_period_c["gse_x_satellite"] = (
    df_desenvolvimento_period_c["gse_x_satellite"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_c["gse_x_satellite"] = (
    df_desenvolvimento_period_c["gse_x_satellite"]
    .apply(pd.to_numeric, downcast ="float"))

df_desenvolvimento_period_c["gse_y_satellite"] = (
    df_desenvolvimento_period_c["gse_y_satellite"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_c["gse_y_satellite"] = (
    df_desenvolvimento_period_c["gse_y_satellite"]
    .apply(pd.to_numeric, downcast ="float"))

df_desenvolvimento_period_c["gse_z_satellite"] = (
    df_desenvolvimento_period_c["gse_z_satellite"]
    .interpolate(method='linear', limit_direction='both'))
df_desenvolvimento_period_c["gse_z_satellite"] = (
    df_desenvolvimento_period_c["gse_z_satellite"]
    .apply(pd.to_numeric, downcast ="float"))

##### Preenchendo Valores ausentes das manchas solares com interpolação linear

##### Preenchendo valores ausentes de Dst com interpolação linear

##### Exportando bases de dados de desenvolvimento

In [ ]:
df_desenvolvimento_period_a.head()

In [ ]:
df_desenvolvimento_period_a.isna().sum()

In [ ]:
df_desenvolvimento_period_a.to_parquet(
    r'..\data\df_desenvolvimento_period_a.parquet')
df_desenvolvimento_period_b.to_parquet(
    r'..\data\df_desenvolvimento_period_b.parquet')
df_desenvolvimento_period_c.to_parquet(
    r'..\data\df_desenvolvimento_period_c.parquet')

### Removendo valores ausentes para base de dados de validação

##### Segmentando base de dados por período

In [ ]:
df_validacao = df_solar_wind.copy()
df_validacao.dropna(how='any', inplace=True)

##### Exportando bases de dados de validação

In [ ]:
df_validacao.to_parquet( r'..\data\df_validacao.parquet')